# Training individual models

## Imports

In [1]:
# CUDA_VISIBLE_DEVICES=0,1
# CUDA_LAUNCH_BLOCKING=1
import os
import time
import numpy as np
from tqdm import tqdm
import rdflib as rl
import torch
import torchtuples as tt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.preprocessing import MinMaxScaler
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
from torch_geometric.data import Data, DataLoader, Batch
from torch_geometric.nn import GCNConv, SAGEConv, GraphConv, GENConv, GATConv
from SAGPooling import SAGPooling
from torch_geometric.nn import global_max_pool as gmp
from torch.nn.parallel import DistributedDataParallel as DDP
import click as ck
import gzip
import pickle
import sys
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import random
# from torch.utils.tensorboard import SummaryWriter

## Constant variables

In [37]:
# Manually categorized cancer subtypes
CANCER_SUBTYPES = [
    [0,12,7,14,4,1,6,2,3],
    [4],
    [5,4,14,6],
    [6,4,12,7],
    [4],
    [6,4,12,7],
    [8],
    [6,4,12],
    [9],
    [6],
    [4],
    [4],
    [4],
    [10],
    [9],
    [4],
    [4,11,12],
    [6],
    [13],
    [12],
    [0,4,12,14],
    [15],
    [4,0,12],
    [4,12],
    [16,17,18,19,20],
    [20],
    [4,12],
    [22],
    [4,14],
    [23],
    [4,12,14],
    [24],
    [21]
]

CELL_TYPES = [
    0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 3, 0, 0, 4, 2, 0,
    0, 0, 5, 0, 0, 6, 0, 0, 7, 8, 0, 9, 0, 0, 0, 0,
    8]

# cancer_types = [
#     "TCGA-ACC", "TCGA-BLCA", "TCGA-BRCA", "TCGA-CESC",
#     "TCGA-CHOL", "TCGA-COAD", "TCGA-DLBC", "TCGA-ESCA",
#     "TCGA-GBM", "TCGA-HNSC", "TCGA-KICH", "TCGA-KIRC",
#     "TCGA-KIRP", "TCGA-LAML","TCGA-LGG","TCGA-LIHC",
#     "TCGA-LUAD","TCGA-LUSC","TCGA-MESO","TCGA-OV",
#     "TCGA-PAAD","TCGA-PCPG","TCGA-PRAD","TCGA-READ",
#     "TCGA-SARC","TCGA-SKCM","TCGA-STAD","TCGA-TGCT",
#     "TCGA-THCA","TCGA-THYM","TCGA-UCEC","TCGA-UCS","TCGA-UVM"]

# cancer_types = [str(sys.argv[1])]

cancer_types = ['TCGA-BRCA']

## Load proteins and interactions

In [38]:
device = 'cuda:0'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def setup(rank, world_size):
#     os.environ['MASTER_ADDR'] = 'localhost'
#     os.environ['MASTER_PORT'] = '12355'

#     # initialize the process group
#     dist.init_process_group("gloo", rank=rank, world_size=world_size)

# def cleanup():
#     dist.destroy_process_group()

# torch.cuda.set_device(device)

proteins_df = pd.read_pickle('data/proteins_700.pkl')
# interactions_df = pd.read_pickle('data/interactions_700.pkl')
interactions_df = pd.read_csv('pr.tsv', header=None, sep='\t')
interactions_df.columns = ['protein1','protein2']
proteins = {row.proteins: row.ids for row in proteins_df.itertuples()}
edge_index = [interactions_df['protein1'].values, interactions_df['protein2'].values]
edge_index = torch.LongTensor(edge_index).to(device)

# edge_index = torch.empty_like(edge_index)
# proteins, edge_index

In [39]:
edge_index

tensor([[    0,     1,     2,  ..., 17182, 17183, 17184],
        [    0,     1,     2,  ..., 17182, 17183, 17184]], device='cuda:0')

In [40]:
# device = torch.device('cpu')

# cancer_type_vector = np.zeros((33,), dtype=np.float32)
# cancer_type_vector[cancer_type] = 1

# cancer_subtype_vector = np.zeros((25,), dtype=np.float32)
# for i in CANCER_SUBTYPES[cancer_type]:
#     cancer_subtype_vector[i] = 1

# anatomical_location_vector = np.zeros((52,), dtype=np.float32)
# anatomical_location_vector[0] = 1
# cell_type_vector = np.zeros((10,), dtype=np.float32)
# cell_type_vector[CELL_TYPES[cancer_type]] = 1

# pt_tensor_cancer_type = torch.FloatTensor(cancer_type_vector).to(device)
# pt_tensor_cancer_subtype = torch.FloatTensor(cancer_subtype_vector).to(device)
# pt_tensor_anatomical_location = torch.FloatTensor(anatomical_location_vector).to(device)
# pt_tensor_cell_type = torch.FloatTensor(cell_type_vector).to(device)
# edge_index = torch.LongTensor(edge_index).to(device)


## Model class

In [41]:
class MyNet(nn.Module):
    def __init__(self, num_nodes, edge_index):
        super(MyNet, self).__init__()
        self.num_nodes = num_nodes
        self.edge_index = edge_index
        self.conv1 = GraphConv(1, 1)
        self.pool1 = SAGPooling(1, ratio=0.1, GNN=GraphConv)
        self.fc1 = nn.Linear(1, 1, bias=False)
        self.bn1 = nn.BatchNorm1d(1)
        self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(1024, 512, bias=False)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(512, 1, bias=False)
        self.bn3 = nn.BatchNorm1d(1)
        self.dropout3 = nn.Dropout(0.1)
        self.sigmoid = nn.Sigmoid()
        self.batches = {}

    def forward(self, data):
#         j = torch.cuda.memory_stats('cuda:0')
#         print(j['allocated_bytes.all.current'],len(self.batches))
        batch_size = data.shape[0]
        x = data[:, :self.num_nodes * 1]
        x = x.reshape(batch_size, self.num_nodes, 1)
        batch=''
        if True:
            l = []
            for i in range(batch_size):
                l.append(Data(x=x[i], edge_index=self.edge_index))
#             if batch_size not in self.batches:
#                 print(x[i])
#                 print(self.edge_index)
            batch = Batch.from_data_list(l)
            self.batches[batch_size] = True
        batch = batch.to(device)
        x = x.to(device)
        x = x.reshape(-1, 1)
        x = F.relu(self.conv1(x=x, edge_index=batch.edge_index))
        x, edge_index, _, batch, perm, score = self.pool1(
            x, batch.edge_index, None, batch.batch)
        x = gmp(x, batch)
        x = x.view(batch_size, -1)
        x = self.dropout1(self.bn1(torch.relu(self.fc1(x))))
#         x = self.dropout2(self.bn2(torch.relu(self.fc2(x))))
#         x = self.dropout3(self.bn3(self.fc3(x)))
        x = self.sigmoid(x)
        return x
    
net = MyNet(len(proteins), edge_index).to(device)
net

MyNet(
  (conv1): GraphConv(1, 1)
  (pool1): SAGPooling(GraphConv, 1, ratio=0.1, multiplier=1.0)
  (fc1): Linear(in_features=1, out_features=1, bias=False)
  (bn1): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (fc2): Linear(in_features=1024, out_features=512, bias=False)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.1, inplace=False)
  (fc3): Linear(in_features=512, out_features=1, bias=False)
  (bn3): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.1, inplace=False)
  (sigmoid): Sigmoid()
)

In [42]:
def normalize(data, minx=None, maxx=None):
    if minx is None:
        minx = np.min(data)
        maxx = np.max(data)
    if minx == maxx:
        return data
    return (data - minx) / (maxx - minx)
        
def normalize_by_row(data):
    for i in range(data.shape[0]):
        data[i, :] = normalize(data[i, :])
    return data

def normalize_by_column(data):
    for i in range(data.shape[1]):
        data[:, i] = normalize(data[:, i])
    return data

# def normalize_by_row_and_column(data):
#     data[:, len(proteins)*0:len(proteins)*6] = normalize_by_row(data[:, len(proteins)*0:len(proteins)*6])
#     data[:, len(proteins)*6:len(proteins)*12] = normalize_by_column(data[:, len(proteins)*6:len(proteins)*12])
#     return data

# def normalize_by_row_and_column_and_matrix(data):
#     data[:, len(proteins)*0:len(proteins)*6] = normalize_by_row(data[:, len(proteins)*0:len(proteins)*6])
#     data[:, len(proteins)*6:len(proteins)*12] = normalize_by_column(data[:, len(proteins)*6:len(proteins)*12])
#     data[:, len(proteins)*12:len(proteins)*18] = normalize(data[:, len(proteins)*12:len(proteins)*18])
#     return data

In [43]:
# def normalize_train(data):
#     minx = np.min(data)
#     maxx = np.max(data)
#     return (data - minx) / (maxx - minx), minx, maxx

# def normalize(data, minx, maxx):
#     return (data - minx) / (maxx - minx)
        
# def normalize_by_row(data):
#     for i in range(data.shape[0]):
#         data[i, :] = normalize(data[i, :])
#     return data

# def normalize_by_column(data):
#     for i in range(data.shape[1]):
#         data[:, i] = normalize(data[:, i])
#     return data

# # def normalize_by_row_and_column(data):
# #     data[:, len(proteins)*0:len(proteins)*6] = normalize_by_row(data[:, len(proteins)*0:len(proteins)*6])
# #     data[:, len(proteins)*6:len(proteins)*12] = normalize_by_column(data[:, len(proteins)*6:len(proteins)*12])
# #     return data

# # def normalize_by_row_and_column_and_matrix(data):
# #     data[:, len(proteins)*0:len(proteins)*6] = normalize_by_row(data[:, len(proteins)*0:len(proteins)*6])
# #     data[:, len(proteins)*6:len(proteins)*12] = normalize_by_column(data[:, len(proteins)*6:len(proteins)*12])
# #     data[:, len(proteins)*12:len(proteins)*18] = normalize(data[:, len(proteins)*12:len(proteins)*18])
# #     return data

In [44]:
# def normalize(data):
#     data_mean = np.mean(data)
#     data_std = np.std(data)
#     if data_std == 0:
#         print('mean = ',data_mean)
#         print('std = ',data_std)
# #     if minx is None:
# #         minx = np.min(data)
# #         maxx = np.max(data)
# #     if minx == maxx:
# #         return data
#         print('results = ',(data - data_mean) / (data_std))
#     return (data - data_mean) / (data_std)


# # def normalize(data, data_mean, data_std):
# # #     data_mean = np.mean(data)
# # #     data_std = np.std(data)
# # #     if data_std == 0:
# # #         print('mean = ',data_mean)
# # #         print('std = ',data_std)
# # #     if minx is None:
# # #         minx = np.min(data)
# # #         maxx = np.max(data)
# # #     if minx == maxx:
# # #         return data
# # #         print('results = ',(data - data_mean) / (data_std))
# #     return (data - data_mean) / (data_std)
        
# def normalize_by_row(data):
#     for i in range(data.shape[0]):
#         data[i, :] = normalize(data[i, :])
#     return data

# def normalize_by_column(data):
#     for i in range(data.shape[1]):
#         data[:, i] = normalize(data[:, i])
#     return data

In [45]:
# lr_ = 0.01
# pat_ = 10
# norm_ = normalize

# # one = 0
# # two = 1
# for i, cancer_type in enumerate(cancer_types):
#     # Create a new model for each cancer type

#     df = pd.read_pickle(f'preprocessing_codes/{cancer_type}.pkl')
# #     print(df)

#     dataset = np.stack(df['features'].values).reshape(len(df), -1)
#     print(dataset.shape)

#     d = []
#     for j in range(17185):
#         print(j)
#         dat = dataset[:, (6*j)+2]
#         print(dat.shape)
#         print(dat)
#         dat = np.log(dat + 1)
#         print(dat)
#         d.append(dat)
# #         print(type(d))

# #     print(dataset)
#     print('meth_data: min = ', np.min(d), 'max = ', np.max(d))

## Train a model for each cancer type

In [47]:
# lr_ = float(sys.argv[2])
# pat_ = int(sys.argv[3])
# norm_ = sys.argv[4]

# one = int(sys.argv[5])
# two = int(sys.argv[6])

lr_ = 0.0001
pat_ = 10
norm_ = normalize_by_row

# one = 0
# two = 1
for i, cancer_type in enumerate(cancer_types):
    # Create a new model for each cancer type

    df = pd.read_pickle(f'preprocessing_codes/{cancer_type}.pkl')
#     print(df)

    dataset = np.stack(df['features'].values).reshape(len(df), -1)
#     print(dataset.shape)
    
#     for j in range(17185):
#         dataset[:, (6*j)+2] = np.log(dataset[:, (6*j)+2] + 1)
#         dataset[:, (6*j)+3] = np.log(dataset[:, (6*j)+3] + 1)
#         dataset[:, (6*j)+5] = np.log(dataset[:, (6*j)+5] + 14)


    dataset = dataset[:, len(proteins)*2:len(proteins)*3]
#     print(type(dataset))
#     result = np.argwhere(dataset == 0)
#     print(result)
#     print(dataset.shape)
#     print('meth_data: min = ', dataset.min(axis=0), 'max = ', dataset.max(axis=0))
#     dataset = np.concatenate((dataset, dataset, dataset), axis=1)

#     dataset = np.concatenate((dataset, dataset), axis=1)

#                 print(dataset.shape)

    in_features = dataset.shape[1]
    labels_days = df['duration'].values
    labels_surv = df['survival'].values

    censored_index = []
    uncensored_index = []
    for i in range(len(dataset)):
        if labels_surv[i] == 1:
            censored_index.append(i)
        else:
            uncensored_index.append(i)
#                 print('Censored', len(censored_index))
#                 print('Uncensored', len(uncensored_index))

    censored_index = np.array(censored_index)
    uncensored_index = np.array(uncensored_index)

#     print(dataset)
#     print(dataset.shape)

    ev_ = []
    splits = 5
    best_cindex = 0

    num_features = 1
    num_nodes = 17185

    for fold in range(splits):
        del net
        torch.manual_seed(0)
        net = MyNet(len(proteins), edge_index).to(device)
#         net = tt.practical.MLPVanilla(in_features, [1024, 512], 1, True, lr_, output_bias=False)
        model = CoxPH(net, tt.optim.Adam(lr_))
        # Censored split
        n = len(censored_index)
        index = np.arange(n)
        i = n // 5
        np.random.seed(seed=0)
        np.random.shuffle(index)
        if fold < 4:
            ctest_idx = index[fold * i: fold * i + i]
            ctrain_idx = np.concatenate((index[:fold * i],index[fold * i + i:]))
        else:
            ctest_idx = index[fold * i:]
            ctrain_idx = index[:fold * i]
        ctrain_n = len(ctrain_idx)
        cvalid_n = ctrain_n // 10
        cvalid_idx = ctrain_idx[:cvalid_n]
        ctrain_idx = ctrain_idx[cvalid_n:]

        # Uncensored split
        n = len(uncensored_index)
        index = np.arange(n)
        i = n // 5
        np.random.seed(seed=0)
        np.random.shuffle(index)
        if fold < 4:
            utest_idx = index[fold * i: fold * i + i]
            utrain_idx = np.concatenate((index[:fold * i],index[fold * i + i:]))
        else:
            utest_idx = index[fold * i:]
            utrain_idx = index[:fold * i]
        utrain_n = len(utrain_idx)
        uvalid_n = utrain_n // 10
        uvalid_idx = utrain_idx[:uvalid_n]
        utrain_idx = utrain_idx[uvalid_n:]


        train_idx = np.concatenate((
            censored_index[ctrain_idx], uncensored_index[utrain_idx]))
        np.random.seed(seed=0)
        np.random.shuffle(train_idx)
        valid_idx = np.concatenate((
            censored_index[cvalid_idx], uncensored_index[uvalid_idx]))
        np.random.seed(seed=0)
        np.random.shuffle(valid_idx)
        test_idx = np.concatenate((
            censored_index[ctest_idx], uncensored_index[utest_idx]))
        np.random.seed(seed=0)
        np.random.shuffle(test_idx)

        normalize_func = norm_

        train_data = dataset[train_idx]

        # Normalize by rows (genes)
        train_data = train_data.reshape(-1, num_nodes, num_features)

        for i in range(num_features):
            train_data[:, :, i] = normalize_func(train_data[:, :, i])
#             if i <= 5:
#                 train_data[:, :, i] = normalize_by_column(train_data[:, :, i])
#             elif i > 5 and i <= 11:
#                 train_data[:, :, i] = normalize_by_row(train_data[:, :, i])
#             else:
#                 train_data[:, :, i] = normalize(train_data[:, :, i])

        train_data = train_data.reshape(-1, num_nodes * num_features)

        train_labels_days = labels_days[train_idx]
        train_labels_surv = labels_surv[train_idx]
        train_labels = (train_labels_days, train_labels_surv)

        val_data = dataset[valid_idx]
        val_data = val_data.reshape(-1, num_nodes, num_features)
        for i in range(num_features):
            val_data[:, :, i] = normalize_func(val_data[:, :, i])
#             if i <= 5:
#                 val_data[:, :, i] = normalize_by_column(val_data[:, :, i])
#             elif i > 5 and i <= 11:
#                 val_data[:, :, i] = normalize_by_row(val_data[:, :, i])
#             else:
#                 val_data[:, :, i] = normalize(val_data[:, :, i])

        val_data = val_data.reshape(-1, num_nodes * num_features)

        val_labels_days = labels_days[valid_idx]
        val_labels_surv = labels_surv[valid_idx]

        test_data = dataset[test_idx]
        test_data = test_data.reshape(-1, num_nodes, num_features)
        for i in range(num_features):
            test_data[:, :, i] = normalize_func(test_data[:, :, i])
#             if i <= 5:
#                 test_data[:, :, i] = normalize_by_column(test_data[:, :, i])
#             elif i > 5 and i <= 11:
#                 test_data[:, :, i] = normalize_by_row(test_data[:, :, i])
#             else:
#                 test_data[:, :, i] = normalize(test_data[:, :, i])

        test_data = test_data.reshape(-1, num_nodes * num_features)


        test_labels_days = labels_days[test_idx]
        test_labels_surv = labels_surv[test_idx]
        val_labels = (val_labels_days, val_labels_surv)

#                     print(val_labels)
#                     print('Training data', train_data.shape)
#                     print('Validation data', val_data.shape)
#                     print('Testing data', test_data.shape)
        callbacks = [tt.callbacks.EarlyStopping(patience=pat_)]
        batch_size = 32
        epochs = 100
        val = (val_data, val_labels)
        log = model.fit(
            train_data, train_labels, batch_size, epochs, callbacks, verbose=True,
            val_data=val,
            val_batch_size=batch_size)
        train = train_data, train_labels
        # Compute the evaluation measurements
        _ = model.compute_baseline_hazards(input=train_data, target=train_labels, batch_size=batch_size)
        surv = model.predict_surv_df(input=test_data, batch_size=batch_size)
        ev = EvalSurv(surv, test_labels_days, test_labels_surv)
        result = ev.concordance_td()
        print('Concordance', result)
        ev_.append(result)

#         if result > best_cindex:
#             best_cindex = result

#             np.savetxt('test'+str(fold+1)+'.csv', test_data, delimiter="\t")

#             np.savetxt('test_labels_days'+str(fold+1)+'.csv', test_labels_days, delimiter="\t")

#             np.savetxt('test_labels_surv'+str(fold+1)+'.csv', test_labels_surv, delimiter="\t")

    print(cancer_type)
    print('lr = '+str(lr_))
    print('patience = '+str(pat_))
    print('normalization = '+str(norm_))
    print(str(statistics.mean(ev_))+"["+str(min(ev_))+"-"+str(max(ev_))+"]")

0:	[0s / 0s],		train_loss: 2.5227,	val_loss: 1.7415
1:	[0s / 0s],		train_loss: 1.8153,	val_loss: 1.7271
2:	[0s / 0s],		train_loss: 1.5120,	val_loss: 1.7059
3:	[0s / 0s],		train_loss: 1.3301,	val_loss: 1.6687
4:	[0s / 1s],		train_loss: 1.2343,	val_loss: 1.6077
5:	[0s / 1s],		train_loss: 1.1444,	val_loss: 1.4813
6:	[0s / 1s],		train_loss: 1.0578,	val_loss: 1.3470
7:	[0s / 1s],		train_loss: 0.9734,	val_loss: 1.1768
8:	[0s / 1s],		train_loss: 0.9572,	val_loss: 1.2395
9:	[0s / 1s],		train_loss: 0.9728,	val_loss: 1.3882
10:	[0s / 1s],		train_loss: 0.8613,	val_loss: 1.1267
11:	[0s / 2s],		train_loss: 0.9100,	val_loss: 1.0933
12:	[0s / 2s],		train_loss: 0.7927,	val_loss: 1.1671
13:	[0s / 2s],		train_loss: 0.8425,	val_loss: 1.1059
14:	[0s / 2s],		train_loss: 0.8163,	val_loss: 1.2343
15:	[0s / 2s],		train_loss: 0.7509,	val_loss: 1.1245
16:	[0s / 2s],		train_loss: 0.8455,	val_loss: 1.1316
17:	[0s / 2s],		train_loss: 0.7664,	val_loss: 1.0163
18:	[0s / 2s],		train_loss: 0.7459,	val_loss: 1.1045
19:

In [ ]:
# for name, param in net.named_parameters():
#     print(name, param.device)

In [ ]:
# dev_props = [torch.cuda.get_device_properties(i) for i in [0,1]]
# print(dev_props)

In [ ]:
# torch.cuda.device_count()

In [ ]:
############ When lr = 0.01 and patience = 10 ############
# TCGA-ACC
# 0.6407741233063947[0.5275590551181102-0.7536231884057971]
# TCGA-BLCA
# 0.6786161956925197[0.5468933177022274-0.7576323987538941]
# TCGA-BRCA
# 0.5264407521138064[0.4601997146932953-0.6046188574405279]
# TCGA-CESC
# 0.6089868487326608[0.4536842105263158-0.765495867768595]
# TCGA-CHOL
# 0.9375[0.6875-1.0]
# TCGA-COAD
# 0.5416567327522879[0.45670460168233545-0.6013229104028863]
# TCGA-DLBC
# 0.6031457431457431[0.4222222222222222-0.8571428571428571]
# TCGA-ESCA
# 0.6415943578057707[0.5074626865671642-0.7283950617283951]
# TCGA-GBM
# 0.699952665470347[0.5194805194805194-0.7992565055762082]
# TCGA-HNSC
# 0.7737905917605848[0.6917922948073701-0.83729216152019]
# TCGA-KICH
# 0.6480249817695936[0.5675675675675675-0.775]
# TCGA-KIRC
# 0.6073085521679165[0.5231447465099192-0.7541371158392435]
# TCGA-KIRP
# 0.6151314344759501[0.48286604361370716-0.7481559536354057]
# TCGA-LAML
# 0.8832229258328248[0.8064516129032258-0.9448818897637795]
# TCGA-LGG
# 0.6443044653183997[0.5934823091247672-0.6978891820580475]
# TCGA-LIHC
# 0.6609286641468511[0.58-0.7258215962441315]
# TCGA-LUAD
# 0.6784523994943508[0.6162207357859532-0.7453441295546559]
# TCGA-LUSC
# 0.6422733977564488[0.6197286395306197-0.665785997357992]
# TCGA-MESO
# 0.9577476896625833[0.8701298701298701-1.0]
# TCGA-OV
# 0.8360278828043031[0.7760635811126695-0.8740636704119851]
# TCGA-PAAD
# 0.6975654816040123[0.5770925110132159-0.8082901554404145]
# TCGA-PCPG
# 0.5456235251701532[0.4766355140186916-0.6860759493670886]
# TCGA-PRAD
# 0.5008283644794941[0.44887278582930756-0.5632447296058661]
# TCGA-READ
# 0.5334667030656595[0.39925373134328357-0.6296296296296297]
# TCGA-SARC
# 0.6346771780431426[0.4776978417266187-0.804920913884007]
# TCGA-SKCM
# 0.7337072562089696[0.6738648947951273-0.8325867861142218]
# TCGA-STAD
# 0.7194974133738123[0.5998831775700935-0.7921166306695464]
# TCGA-TGCT
# 0.6178811566240728[0.5053191489361702-0.7293577981651376]
# TCGA-THCA
# 0.5299605129796416[0.4819364161849711-0.5824175824175825]
# TCGA-THYM
# 0.5782367078355151[0.44-0.7461538461538462]
# TCGA-UCEC
# 0.6120250711669957[0.5395114942528736-0.6941176470588235]
# TCGA-UCS
# 0.7368138376632117[0.6086956521739131-0.8823529411764706]
# TCGA-UVM
# 0.681177536231884[0.5625-0.8]

In [ ]:
# ############ When lr = 0.0001 and patience = 20 ############
# TCGA-ACC
# 0.6882712329658379[0.6376811594202898-0.7971014492753623]
# TCGA-BLCA
# 0.5890027508380132[0.5258245177349098-0.6554517133956387]
# TCGA-BRCA
# 0.527074110092128[0.4387621540762902-0.5636395207501302]
# TCGA-CESC
# 0.589973099622829[0.540506329113924-0.712468193384224]
# TCGA-CHOL
# 0.5555555555555556[0.0-1.0]
# TCGA-COAD
# 0.5459434785976518[0.4670074349442379-0.6057171020206998]
# TCGA-DLBC
# 0.4984848484848485[0.25-0.9090909090909091]
# TCGA-ESCA
# 0.4986375992316221[0.4444444444444444-0.6232394366197183]
# TCGA-GBM
# 0.6782654586317148[0.6045627376425855-0.7610294117647058]
# TCGA-HNSC
# 0.6791943114955646[0.6413373860182371-0.7038001041124414]
# TCGA-KICH
# 0.6620646816259077[0.41304347826086957-0.875]
# TCGA-KIRC
# 0.6227483865794052[0.5598824393828068-0.671559633027523]
# TCGA-KIRP
# 0.4827822329411887[0.4236453201970443-0.598937583001328]
# TCGA-LAML
# 0.8323271507079413[0.7612903225806451-0.8620689655172413]
# TCGA-LGG
# 0.526960884964874[0.4564932225623087-0.6001838235294118]
# TCGA-LIHC
# 0.5697073188251252[0.5463623395149786-0.6169014084507042]
# TCGA-LUAD
# 0.5905062967495905[0.5593645484949833-0.6244131455399061]
# TCGA-LUSC
# 0.5712931058147199[0.5031739314430809-0.6450902686041391]
# TCGA-MESO
# 0.7864640324214792[0.6818181818181818-0.8181818181818182]
# TCGA-OV
# 0.703446385128956[0.636109818520242-0.7775675675675676]
# TCGA-PAAD
# 0.6147314269758151[0.5191082802547771-0.7631578947368421]
# TCGA-PCPG
# 0.5002996900088952[0.3911764705882353-0.6241830065359477]
# TCGA-PRAD
# 0.4923555066084578[0.4510086455331412-0.5229147571035747]
# TCGA-READ
# 0.5850391331932935[0.5111940298507462-0.6925925925925925]
# TCGA-SARC
# 0.6605232504700553[0.6114982578397212-0.735973597359736]
# TCGA-SKCM
# 0.6151867750657317[0.5194877505567929-0.7043673012318029]
# TCGA-STAD
# 0.6225493511385155[0.5163551401869159-0.7319899244332494]
# TCGA-TGCT
# 0.584073905256942[0.5478723404255319-0.6246851385390428]
# TCGA-THCA
# 0.4994043519200372[0.3592269799166351-0.6006351726875744]
# TCGA-THYM
# 0.5762585579099554[0.424-0.8153846153846154]
# TCGA-UCEC
# 0.5615907273146579[0.5028735632183908-0.6479799561540871]
# TCGA-UCS
# 0.790509585985424[0.6521739130434783-0.9565217391304348]
# TCGA-UVM
# 0.6054166666666666[0.525-0.7]